In [1]:
import pandas as pd
import torch
import numpy as np
import scipy
from sklearn import metrics
import matplotlib.pyplot as plt
import random
from sklearn.metrics import classification_report
from transformers import GenerationConfig
import torch

In [2]:
from transformers.utils import logging
logging.get_logger("transformers").setLevel(logging.ERROR)

In [3]:
data_path = './empathic-stories/data/'

In [4]:
train_df = pd.read_csv(f'{data_path}/PAIRS (train).csv')
dev_df = pd.read_csv(f'{data_path}/PAIRS (dev).csv')
test_df = pd.read_csv(f'{data_path}/PAIRS (test).csv')

In [5]:

label_columns = "similarity_empathy_human_AGG	similarity_event_human_AGG	similarity_emotion_human_AGG	similarity_moral_human_AGG".split()[::-1]
text_preprocess = lambda x: x.strip().replace("\n", " ")

In [6]:
import trl

In [7]:
from trl.commands.cli_utils import SftScriptArguments

In [8]:
story_in_use = 'summary'
label_in_use = 'empathy'

In [9]:

text_columns = {
    "summary": ["story_A_summary", "story_B_summary"],
    "full": ["story_A", "story_B"],
}.get(story_in_use)
label_column = {
    "empathy": ["similarity_empathy_human_AGG"],
    "event": ["similarity_event_human_AGG"],
    "emotion": ["similarity_emotion_human_AGG"],
    "moral": ["similarity_moral_human_AGG"],
    "all": label_columns,
}.get(label_in_use)

In [10]:
score_conversion_funcs = {
    "none": lambda x:x,
    "original_paper": lambda x: x / 4,
    "01_continue": lambda x:(x - 1) / 3,
}
score_recover_funcs = {
    "none": lambda x:x,
    "original_paper": lambda x: x * 4,
    "01_continue": lambda x:(x * 3) + 1,
}

In [11]:
# Choose score conversion method
score_conversion_in_use = "none"

In [12]:
score_conversion_func = score_conversion_funcs.get(score_conversion_in_use)
score_recover_func = score_recover_funcs.get(score_conversion_in_use)

In [13]:

def create_data(df, text_pps, score_conversion_funcs):
    required_columns = text_columns + label_column
    score_names = [f"score_{i}" for i in range(len(label_column))]
    df = df[required_columns].rename(
        columns={
            k: v
            for k, v in zip(
                required_columns, ["sentence1", "sentence2"] + score_names
            )
        }
    )
    for i in [1, 2]:
        df[f"sentence{i}"] = df[f"sentence{i}"].apply(text_pps)
    for i in range(len(label_column)):
        df[f"score_{i}"] = score_conversion_funcs(df[f"score_{i}"])
    return df

In [14]:
train_df = create_data(train_df, text_preprocess, score_conversion_func)
dev_df = create_data(dev_df, text_preprocess, score_conversion_func)
test_df = create_data(test_df, text_preprocess, score_conversion_func)

In [15]:
import datasets
from datasets import Dataset

In [16]:
optimized_similarity_system_prompt = """Rate the extent to which you agree with the statement "the narrators of the two stories would empathize with each other." We define empathy as feeling, understanding, and relating to what another person is experiencing. Note that it is possible to have empathy even without sharing the exact same experience or circumstance. Importantly, for two stories to be empathetically similar, both narrators should be able to empathize with each other (if narrator A’s story was shared in response to narrator B’s story, narrator B would empathize with narrator A and vice versa). Give your answer on a scale from 1-4 (1-not at all, 2-not so much, 3-very much, 4-extremely), with 0.5 increments in each level between 1-4 are allowed. Please only return the score without any explanation.""".strip()

In [17]:

optimized_similarity_user_prompt = """
### Narrative A:
{story_a}

### Narrative B:
{story_b}

### Similarity Score:
"""

In [18]:
from transformers import AutoTokenizer

In [19]:
tokenizer = AutoTokenizer.from_pretrained("./fullpsft_llama3-emp_2gpus-summary-full_text_loss/", padding_side='left')

In [20]:

def promptify(x1, x2, labels):
    user_input = optimized_similarity_user_prompt.format(
        story_a=x1, story_b=x2
    ).strip()
    label = ' '.join([str(score_conversion_func(l)) for l in labels])
    model_input = tokenizer.apply_chat_template(
        [
            {"role": "system", "content": optimized_similarity_system_prompt},
            {"role": "user", "content": user_input},
            {"role": "assistant", "content": label},
        ],
        tokenize=False,
    )
    return {"text": model_input}

def create_dataset(df, concat_reverse=False, shuffle=False):
    score_columns = [f"score_{i}" for i in range(len(label_column))]
    dataset = (
        Dataset.from_pandas(df)
        .map(lambda x: promptify(x["sentence1"], x["sentence2"], [x[s] for s in score_columns]))
        .remove_columns(["sentence1", "sentence2"] + score_columns)
    )
    if concat_reverse:
        _ = (
            Dataset.from_pandas(df)
            .map(lambda x: promptify(x["sentence2"], x["sentence1"], [x[s] for s in score_columns]))
            .remove_columns(["sentence1", "sentence2"] + score_columns)
        )
        dataset = datasets.concatenate_datasets([dataset, _])
    if shuffle:
        dataset = dataset.shuffle()
    return dataset


In [21]:
train_dataset = create_dataset(train_df, concat_reverse=True, shuffle=True)
dev_dataset = create_dataset(dev_df, concat_reverse=False, shuffle=False)
test_dataset = create_dataset(test_df, concat_reverse=False, shuffle=False)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [22]:
from transformers import AutoModelForCausalLM
from peft import AutoPeftModelForCausalLM

In [23]:
model = AutoModelForCausalLM.from_pretrained('./fullpsft_llama3-emp_2gpus-summary-full_text_loss/')

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [24]:
DEVICE='cuda:1'

In [25]:
model = model.to(DEVICE).to(torch.bfloat16).eval()
model.generation_config.pad_token_id=tokenizer.eos_token_id

In [31]:
generation_config = GenerationConfig.from_pretrained('meta-llama/Meta-Llama-3-8B-Instruct')
tokenizer.pad_token_id = tokenizer.added_tokens_encoder[
    "<|reserved_special_token_0|>"
]
tokenizer.padding_side = "left"

generation_config.max_length = 512

generation_config.temperature = 0.0
generation_config.pad_token_id = tokenizer.pad_token_id
generation_config.do_sample = False
model.generation_config.pad_token_id=tokenizer.pad_token_id

In [32]:
def test_predict_batch(samples):
    model_inputs, labels = zip(*[ sample.split('<|start_header_id|>assistant<|end_header_id|>') for sample in samples['text']])
    model_inputs = [ x+'<|start_header_id|>assistant<|end_header_id|>' for x in model_inputs]
    # model_input, label = sample['text'].split('<|start_header_id|>assistant<|end_header_id|>')
    labels = [ label.strip().split('<')[0] for label in labels]
    model_inputs = tokenizer(model_inputs, add_special_tokens=False, return_tensors='pt', padding=True)
    
    response = model.generate(**{ k:v.to(DEVICE) for k,v in model_inputs.items()}, generation_config=generation_config, pad_token_id=tokenizer.eos_token_id)
    return model_inputs, response

In [33]:
# model.config.pad_token_id = tokenizer.eos_token_id

In [34]:
model_inputs, response = test_predict_batch(dev_dataset[:2])

/home/minghanwu/.conda/envs/py38/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/minghanwu/.conda/envs/py38/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [36]:
def predict_batch(samples):
    model_inputs, labels = zip(*[ sample.split('<|start_header_id|>assistant<|end_header_id|>') for sample in samples['text']])
    model_inputs = [ x+'<|start_header_id|>assistant<|end_header_id|>' for x in model_inputs]
    labels = [ label.strip().split('<')[0] for label in labels]
    model_inputs = tokenizer(model_inputs, add_special_tokens=False, return_tensors='pt', padding=True)
    
    response = model.generate(**{ k:v.to(DEVICE) for k,v in model_inputs.items()}, generation_config=generation_config, pad_token_id=tokenizer.eos_token_id)
    predicted_scores = [ tokenizer.decode(response[i][model_inputs.input_ids[i].masked_select(model_inputs.input_ids[i]!=tokenizer.pad_token_id).shape[0]:]).split('\n')[-1].split('<')[0] for i in range(len(labels))]
    predicted_scores = [ multi_scores.split() for multi_scores in predicted_scores]
    labels = [ multi_labels.split() for multi_labels in labels]
    return predicted_scores, labels

In [37]:
# train_df.groupby('score_0').count()

In [38]:
# predict_batch(dev_dataset[:2])

In [39]:
model.generation_config.padding_side='left'

In [40]:
import tqdm

In [41]:
def make_prediction(dataset, bsz=20):
    predictions = []
    labels = []
    for i in tqdm.tqdm(range(len(dataset) // bsz + 1)):
        batch = dataset[i*bsz:(i+1)*bsz]
        if len(batch['text']) == 0:
            break
        predicted_score, label = predict_batch(batch)
        predictions.extend(predicted_score)
        labels.extend(label)
    bin_labels = [ [int(float(x)>2.5) for x in multi_labels] for multi_labels in labels]
    bin_prediction = [ [int(float(x)>2.5) for x in multi_predictions ] for multi_predictions in predictions]
    float_predictions = [ [float(x) for x in multi_predictions ] for multi_predictions in predictions]
    float_labels = [ [float(x) for x in multi_labels ] for multi_labels in labels]
    return float_predictions, float_labels, bin_prediction, bin_labels

In [42]:
# float_predictions, float_labels, bin_prediction, bin_labels = make_prediction(dev_dataset)

In [43]:
def evaluate(dataset):
    float_predictions, float_labels, bin_prediction, bin_labels = make_prediction(dataset)
    cls_dfs = [ pd.DataFrame(classification_report(bin_label, bin_prediction, output_dict=True)).T for bin_label, bin_prediction in zip(zip(*bin_prediction), zip(*bin_labels))]
    spearmanrs = [scipy.stats.spearmanr(np.array(float_label),np.array(float_prediction)).statistic for float_label, float_prediction in zip(zip(*float_predictions), zip(*float_labels))]
    pearsonrs = [ scipy.stats.pearsonr(np.array(float_label),np.array(float_prediction)).statistic  for float_label, float_prediction in zip(zip(*float_predictions), zip(*float_labels))]
    mses = [  np.square(np.subtract(np.array(float_label),np.array(float_prediction))).mean()  for float_label, float_prediction in zip(zip(*float_predictions), zip(*float_labels))]
    sp_df = pd.DataFrame({'S': spearmanrs, 'P': pearsonrs, 'M':mses})
    return cls_dfs, sp_df, float_predictions, float_labels, bin_prediction, bin_labels

In [44]:
dev_cls_df, dev_sp_df, predicted_dev_scores, dev_score, dev_bin_predictions, dev_bin_labels=evaluate(dev_dataset)

 83%|██████████████████████████████████████████████████████████████████████▊              | 5/6 [00:07<00:01,  1.47s/it]


In [45]:
dev_cls_df[0]

,precision,recall,f1-score,support
0,0.596774,0.755102,0.666667,49.00
1,0.684211,0.509804,0.584270,51.00
accuracy,0.630000,0.630000,0.630000,0.63
macro avg,0.640492,0.632453,0.625468,100.00
weighted avg,0.641367,0.630000,0.624644,100.00


In [46]:
print(dev_cls_df[0].to_latex(float_format=lambda x:f'{x:.3f}'))

\begin{tabular}{lrrrr}
\toprule
 & precision & recall & f1-score & support \\
\midrule
0 & 0.597 & 0.755 & 0.667 & 49.000 \\
1 & 0.684 & 0.510 & 0.584 & 51.000 \\
accuracy & 0.630 & 0.630 & 0.630 & 0.630 \\
macro avg & 0.640 & 0.632 & 0.625 & 100.000 \\
weighted avg & 0.641 & 0.630 & 0.625 & 100.000 \\
\bottomrule
\end{tabular}



In [47]:
print(dev_sp_df.rename(columns={'S':'Spearman','P':"Pearson", 'M':'MSE'}).to_latex(float_format=lambda x:f'{x:.3f}'))

\begin{tabular}{lrrr}
\toprule
 & Spearman & Pearson & MSE \\
\midrule
0 & 0.209 & 0.189 & 0.627 \\
\bottomrule
\end{tabular}



In [48]:
test_cls_df, test_sp_df, predicted_test_scores, test_score, test_bin_predictions, test_bin_labels=evaluate(test_dataset)

  0%|                                                                                            | 0/21 [00:00<?, ?it/s]/home/minghanwu/.conda/envs/py38/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/minghanwu/.conda/envs/py38/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
 95%|███████████████████████████████████████████████████████████████████████████████    | 20/21 [00:30<00:01,  1.50s/it]


In [49]:
print(test_cls_df[0].to_latex(float_format=lambda x:f'{x:.3f}'))

\begin{tabular}{lrrrr}
\toprule
 & precision & recall & f1-score & support \\
\midrule
0 & 0.431 & 0.494 & 0.460 & 170.000 \\
1 & 0.580 & 0.517 & 0.547 & 230.000 \\
accuracy & 0.507 & 0.507 & 0.507 & 0.507 \\
macro avg & 0.506 & 0.506 & 0.504 & 400.000 \\
weighted avg & 0.517 & 0.507 & 0.510 & 400.000 \\
\bottomrule
\end{tabular}



In [50]:
test_cls_df[0]

,precision,recall,f1-score,support
0,0.430769,0.494118,0.460274,170.0000
1,0.580488,0.517391,0.547126,230.0000
accuracy,0.507500,0.507500,0.507500,0.5075
macro avg,0.505629,0.505754,0.503700,400.0000
weighted avg,0.516857,0.507500,0.510214,400.0000


In [51]:
print(test_cls_df[0].to_latex(float_format=lambda x:f'{x:.3f}'))

\begin{tabular}{lrrrr}
\toprule
 & precision & recall & f1-score & support \\
\midrule
0 & 0.431 & 0.494 & 0.460 & 170.000 \\
1 & 0.580 & 0.517 & 0.547 & 230.000 \\
accuracy & 0.507 & 0.507 & 0.507 & 0.507 \\
macro avg & 0.506 & 0.506 & 0.504 & 400.000 \\
weighted avg & 0.517 & 0.507 & 0.510 & 400.000 \\
\bottomrule
\end{tabular}



In [53]:
print(test_sp_df.rename(columns={'S':'Spearman','P':"Pearson", 'M':'MSE'}).to_latex(float_format=lambda x:f'{x:.3f}'))

\begin{tabular}{lrrr}
\toprule
 & Spearman & Pearson & MSE \\
\midrule
0 & 0.028 & 0.039 & 0.773 \\
\bottomrule
\end{tabular}



In [46]:
print(test_sp_df.rename(columns={'S':'Spearman','P':"Pearson", 'M':'MSE'}).to_latex(float_format=lambda x:f'{x:.3f}'))

\begin{tabular}{lrrr}
\toprule
 & Spearman & Pearson & MSE \\
\midrule
0 & 0.003 & 0.005 & 0.811 \\
\bottomrule
\end{tabular}

